# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

#compute cluster
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

#dataset
from azureml.core import Dataset
from sklearn.model_selection import train_test_split

#automl
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

#model deployment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice

# Others
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# choose a name for experiment
experiment_name = 'my-capstone'
experiment=Experiment(ws, experiment_name)

Check the experiment

In [ ]:
experiment

### Computer Cluster

Lets create computer cluster first, if we have any existing cluster we will use that one, otherwise we will create a brand new cluster

here, we will use **max_nodes** 6

In [ ]:
# Choosing a name for the CPU cluster
amlcompute_cluster_name = "my-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)

### Create Datasets

IN this project we are going to use **The Heart Failure Dataset** and this datasets can be found in [Kaggle](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data). 

The Heart failure dataset has total 13 columns 
- age
- anaemia
- creatinine_phosphokinase 
- diabetes
- ejection_fraction
- high_blood_pressure
- platelets
- serum_creatinine
- serum_sodium
- sex
- smoking
- time
- DEATH_EVENT.

First, we will create the datasets and then we will register that dataset into the workspace

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

url = "https://raw.githubusercontent.com/Iamsdt/Udacity-ML-Azure-Capstone/master/datasets/heart_failure_clinical_records_dataset.csv"

dataset = Dataset.Tabular.from_delimited_files(example_data)        

#Register Dataset in Workspace
key = "Bank-marketing"
description_text = "Bank Marketing DataSet for Udacity Course 2"
dataset = dataset.register(workspace=ws, name=key, description=description_text)

# convert into pandas
df = dataset.to_pandas_dataframe()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In **AutoML**, I used following parameters



| No.  | Title                       | Value          |
| ---- | --------------------------- | -------------- |
| 1.   | Task                        | Classification |
| 2.   | Primary Metric              | Accuracy       |
| 3.   | Number of cross validations | 5              |
| 4.   | Experiment Time out minutes | 20             |

Explanation:
- **task** -> **classificaion**, we want to determine value of DEATH_EVENT based on the other 12 features
- **primary_metric** -> **accuracy**, we choose accuracy as our primary metrics
- **max_concurrent_iterations** -> **5**, in our cluster, we have maximum 6 nodes, so we can use 5

In [ ]:
# automl settings
automl_settings = {
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 5,
}

# folder location
project_folder = './heart-failure'

# automl config
automl_config = AutoMLConfig(task = 'classification',
                            experiment_timeout_minutes=20,
                            compute_target = compute_target,
                            training_data = df,
                            label_column_name = 'DEATH_EVENT',
                            featurization= 'auto',
                            path = project_folder,
                            debug_log = "automl_errors.log",
                            **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

In [ ]:
# Lets wait
remote_run.wait_for_completion(show_output = True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
best_run

Check the metrics from the best run

In [ ]:
best_run.get_metrics()

Checkout best model

In [ ]:
fitted_model

In [ ]:
fitted_model.steps[-1][-1].estimators[0]

Check best metrics, using **get_best_child()** methods

In [ ]:
auto_best_run = remote_run.get_best_child()

auto_best_run.get_metrics()

Register the model in the workspace

In [ ]:
#TODO: Save the best model
model_name = best_run.properties['model_name']
# register model in workspace
mymodel = remote_run.register_model(model_name=model_name, description='Best automml model heart failure model', tags=None)

In [ ]:
mymodel

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [ ]:
inference_config = InferenceConfig(entry_script='score.py', environment=best_run.get_environment())

In [ ]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=1, auth_enabled=True, enable_app_insights=True)

deployed_service = Model.deploy(workspace=ws, 
                       name='automl-best-model',
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       models=[mymodel])

In [ ]:
deployed_service.wait_for_deployment(show_output = True)

TODO: In the cell below, send a request to the web service you deployed to test it.

Let's time for communicate with the deployed server,
we will use python **urllib** library to communicate with the server

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

data = {
    "data":
    [
        {
            'Column1': "0",
            'age': "75",
            'anaemia': "2",
            'creatinine_phosphokinase': "500",
            'diabetes': "1",
            'ejection_fraction': "50",
            'high_blood_pressure': "1",
            'platelets': "127000",
            'serum_creatinine': "130",
            'serum_sodium': "0",
            'sex': "1",
            'smoking': "1",
            'time': "4",
        }
    ],
}

body = str.encode(json.dumps(data))

url = 'http://152628f3-a8cb-4116-a091-3f7c2611d3cb.southcentralus.azurecontainer.io/score'
api_key = 'btWNaLY4LjCbuARcXgBrluF6bV2AAdGQ' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
deployed_service.get_logs()

## Clean up

Delete the deployed service

In [ ]:
deployed_service.delete()

Delete computer cluster

In [ ]:
compute_target.delete()